In [1]:
import numpy as np
import string, math, random
from copy import deepcopy
from tqdm import tqdm

In [2]:
def distancesFromCoords():
    f = open('kroA100.tsp')
    data = [line.replace("\n","").split(" ")[1:] for line in f.readlines()[6:106]]
    coords =  list(map(lambda x: [float(x[0]),float(x[1])], data))
    distances = []
    for i in range(len(coords)):
        row = []
        for j in range(len(coords)):
            row.append(math.sqrt((coords[i][0]-coords[j][0])**2 + (coords[i][1]-coords[j][1])**2))
        distances.append(row)
    return distances

In [3]:
def localHeuristics(matriz,col,row):
    matrizHL = deepcopy(matriz)
    for i in range(row):
        for j in range(col):
            if(matrizHL[i][j] != 0 ):
                matrizHL[i][j] = 1/matrizHL[i][j]
    return np.array(matrizHL)

In [4]:
def ObjetiveFunction(route,distances): 
    sum=0
    for i in range(len(route)-1): 
        fromCity=route[i]
        toCity=route[i+1]
        sum=sum+distances[fromCity][toCity]
    return sum

In [5]:
def generateRoutes(nCities, pos, matriz):

    aux_list=[]
    for i in range(nCities):
        if(i!=pos):
            aux_list.append(i)

    route=[pos]
    while len(aux_list)>0:
        aux = random.choice(aux_list)
        aux_list.remove(aux)
        route.append(aux)
    route.append(pos)
    z = ObjetiveFunction(route,matriz)

    return route,z

In [6]:
def initialPheromone(matriz, n, pos):
    pheromone = np.zeros(np.shape(matriz))
    while(n>0):
        nSolution, z = generateRoutes(len(matriz),pos, matriz)
        inverseZ=1/z
        for i in range(len(nSolution)-1):
            fromCity=nSolution[i]
            toCity=nSolution[i+1]
            pheromone[fromCity][toCity]+=inverseZ

        n-=1
    return pheromone


In [7]:
def evaporation(pheromone):
    p = 0.1
    for i in range(len(pheromone)):
        for j in range(len(pheromone[0])):
            pheromone[i][j] = pheromone[i][j]*(1-p)
    return pheromone

In [8]:
def Probabilities(local_heu,pheromone,alpha,beta,positions):

    matriz=(pheromone**alpha)*(local_heu**beta)

    for i in positions:
        matriz[i,:]=0
        
    summation=matriz.sum(axis=0)

    probabilities=matriz/summation
    return probabilities

In [9]:
def accumulatedPositions(probability,ran,positions):
    accumulated=0
    for i in range(len(probability)):
        accumulated+=probability[i]
        if probability[i] in positions:
            continue
        else:
            if(accumulated>ran):
                return i
    return 0

In [10]:
def addPheromone(pheromone,solution,distances):
    aux_pheromone = deepcopy(pheromone)
    z = ObjetiveFunction(solution,distances)
    inverse_z = 1/z
    for i in range(len(solution)-1):
        fromCity = solution[i]
        toCity = solution[i+1]
        aux_pheromone[fromCity][toCity] += inverse_z
    
    return np.array(aux_pheromone)

In [11]:
def generatePath(local_heu,pheromone,alpha,beta,pos):
    counter=0
    positions=[pos]
    while(len(positions)<len(pheromone)):
        ran=random.random()
        probabilities=Probabilities(local_heu,pheromone,alpha,beta,positions)
        positions.append(accumulatedPositions(probabilities[:,positions[counter]],ran,positions))
        counter+=1
    positions.append(pos)
    return positions

In [17]:
if __name__ == "__main__":

    distances = distancesFromCoords()
    pos = random.randint(0,len(distances)-1)
    cities = len(distances)
    while(True):
        n = input("Number of routes: ")
        try:
            n = int(n)
            if(n<0):
                print("\nInvalid number!\n")
            else:
                break
        except ValueError:
            print("\nInsert a number!\n")
    while(True):
        alpha = input("Alpha: ")
        try:
            alpha = int(alpha)
            if(alpha<0):
                print("\nInvalid number!\n")
            else:
                break
        except ValueError:
            print("\nInsert a number!\n")
    while(True):
        beta = input("Beta: ")
        try:
            beta = int(beta)
            if(beta<0):
                print("\nInvalid number!\n")
            else:
                break
        except ValueError:
            print("\nInsert a number!\n")
    while(True):
        iterations = input("Iterations: ")
        try:
            iterations = int(iterations)
            if(iterations<0):
                print("\nInvalid number!\n")
            else:
                break
        except ValueError:
            print("\nInsert a number!\n")
            
    local_h = localHeuristics(distances,len(distances[0]),cities)
    pheromone_matrix = initialPheromone(distances,n, pos) 
    for i in tqdm(range(iterations)):
        solution = generatePath(local_h,pheromone_matrix,alpha,beta, pos)
        pheromone_matrix = addPheromone(pheromone_matrix,solution,distances)
        pheromone_matrix = evaporation(pheromone_matrix)
        
    print("\nSolution for TSP:")
    print(solution)
    print("z:" ,ObjetiveFunction(solution,distances))

Number of routes: 5000
Alpha: 1
Beta: 5
Iterations: 100


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]c:\users\usuario\miniconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 12.79it/s]



Solution for TSP:
[78, 17, 23, 37, 35, 83, 9, 89, 48, 5, 62, 0, 91, 7, 41, 88, 30, 79, 55, 96, 74, 18, 52, 87, 15, 21, 93, 69, 65, 64, 3, 25, 98, 71, 20, 73, 58, 14, 16, 10, 31, 90, 97, 22, 44, 85, 26, 11, 19, 56, 6, 8, 86, 50, 60, 24, 80, 68, 63, 39, 53, 1, 43, 49, 72, 67, 84, 29, 38, 95, 77, 51, 4, 36, 75, 32, 12, 94, 81, 13, 47, 99, 70, 40, 42, 2, 45, 28, 33, 82, 54, 76, 59, 61, 34, 27, 92, 57, 66, 0, 78]
z: 26465.84724699426
